## Задача 1 ##
Продакт на главной mail.ru решил протестировать в рекомендательной ленте контента вместо карточек со статьями видеоплеер с короткими видео. Нынешний таймспент на юзера в день в среднем равен 25 минут, а стандартная ошибка (SD) равна 156. Мы предполагаем, что в новой версии таймспент на юзера в день изменится на 10%. Средний трафик 20000 человек в день. Посчитайте сколько дней необходимо держать эксперимент при alpha = 5% и beta = 20% .

In [2]:
from typing import Union
from math import asin
from statsmodels.stats.power import tt_ind_solve_power, zt_ind_solve_power

def calc_continuous_effect_size(mean_1: Union[float, int],
                                std_1: Union[float, int],
                                mean_2: Union[float, int],
                                std_2: Union[float, int],
                                nobs_1: int = 10_000,
                                nobs_2: int = 10_000):
    
    es_formula = (mean_1 - mean_2) / ((std_1**2 + std_2**2) / 2) ** 0.5  
    return es_formula

def calc_sample_size_continuous(effect_size: float,
                                alpha: float = .05,
                                beta: float = .2,
                                ratio: Union[float, int] = 1):
    
    n = tt_ind_solve_power(effect_size=effect_size,
                           alpha=alpha,
                           power=(1 - beta),
                           ratio=ratio,
                  )
    return int(n * 2)


mu_control, mu_test = 25, 25*1.10 # Увеличим тестовое среднее по условию задачи на 10 процентов
std_control, std_test = 156, 156*1.10 # Увеличим тестовое СКО на 10 процентов для чистоты эксперимента
traffic_per_day = 20000


effect_size = calc_continuous_effect_size(mean_1=mu_control, std_1=std_control, mean_2=mu_test, std_2=std_test)
traffic = calc_sample_size_continuous(effect_size=effect_size, alpha=.05, beta=.2)

traffic/traffic_per_day

6.7542

**Ответ:** Неделю необходимо держать эксперимент, при условии что мы весь трафик отдали под эксперимент. Я бы выделил месяц, для проведения эксперимента с 25% ежедневного трафика, т.е. примерно 12% пользователей будут получать видео вместо карточек. Опасения связанны с тем, что работа с видео (хранение, организация потока) требует больших мощностей и как поведут сервера компании под большой нагрузкой у меня нет данных. Более того, со стороны пользователей системные требования к ПК или смартфонам, тоже увеличиваются - это может сказаться на репутации компании, т.к. сайт либо будет дольше загружаться, либо в виду отсутствия мощного "железа" пользователь предпочтет конкурентов.

## Задача 2 ## 
Наша продуктовая команда в ecommerce магазине планирует запустить тест, направленный на ускорение загрузки сайта. Одна из основных метрик bounce rate в GA = 40%. Мы предполагаем, что при оптимизации сайта она изменится минимум на 20%.Средний трафик 4000 человек в день. Посчитайте сколько нам нужно дней держать эксперимент при alpha = 5% и beta = 20%

### Примечания ###
1. Не ясны условия задачи. Оптимизация сайта изменится на 20% относительно 40% и составить 32% или bounce rate после оптимизации составит 20%.
2. Предположу, что bounce rate станет после оптимизации 20%.
3. Bounce rate Google Analytics = $\frac{\text{кол-во посетителей покинувших первую же страницу сайта}}{\text{общее количество посетителей в день}}$ следовательно нужно считать рост доли посетителей оставшихся на сайте, т.е. тех кто перешел с первой страницы на другую.

In [28]:
from typing import Union
from math import asin
from statsmodels.stats.power import tt_ind_solve_power, zt_ind_solve_power

def calc_propotion_effect_size(conv_1: int, nobs_1: int, conv_2: int, nobs_2: int):
    prob_1, prob_2 = conv_1 / nobs_1, conv_2 / nobs_2
    es_formula = 2  * asin(prob_1**0.5) - 2 * asin(prob_2**0.5) 
    return es_formula

def calc_sample_size_proportion(effect_size: float,
                                alpha: float = .05,
                                beta: float = .2,
                                ratio: Union[float, int] = 1):
    
    n = zt_ind_solve_power(effect_size=effect_size,
                           alpha=alpha,
                           power=(1 - beta),
                           ratio=ratio,
                  )
    return int(n * 2)

traffic_per_day = 4000
conv_1, conv_2 = 100-40, 100-20 # доли посетителей которые не ушли с первой страницы сайта
one_page_users = 40*4000/100 # количество пользователей которые покидают первую страницу
nobs_1, nobs_2 = 100, 100 # количество наблюдений для того чтобы получились доли 0.6 0.8
prob_1, prob_2 = conv_1 / nobs_1, conv_2 / nobs_2 
# print(prob_1, prob_2)
effect_size = calc_propotion_effect_size(conv_1=conv_1, nobs_1=nobs_1, conv_2=conv_2, nobs_2=nobs_2)
size = calc_sample_size_proportion(effect_size=effect_size, alpha=.05, beta=.2)
print(f"Рекомендуемая выборка {size}, количество юзеров покидающих первую страницу в день {one_page_users:.0f}")

Рекомендуемая выборка 160, количество юзеров покидающих первую страницу в день 1600


Ответ: для эксперимента достаточно одного дня со случайной выборкой в 160 юзеров, что составляет 10% от всех "отказников" за сутки, чтобы понять, что их количество сократилось до 20% процентов